# This code is intended to be used for offline analysis of the magnetic data retreived from the WebDAQ during site surveying.

## You may need to update the widget package to make this work.

## pip install ipywidgets --upgrade
## conda install conda-forge::ipywidgets

## Do not run all
## Run one cell at a time or plots may not populate

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
from ipywidgets import FloatSlider
from ipywidgets import Layout 
from scipy import signal
from matplotlib import gridspec
from scipy.fft import fftshift
import warnings

############################# If you are using Mac, you may need to remove the r before the path ###############################
mag = pd.read_csv(r"Put\data\path\here", skiprows = 1, delimiter=',')  
                 # Keep the quotation marks
mag.columns = ["Sample", "Time (s)", "Noise (V)", "Voltage Z (V)", "Voltage Y (V)", "Voltage X (V)", "blank"]

samples = mag['Sample']
times = mag['Time (s)']
noi = mag['Noise (V)']
k = mag['Voltage Z (V)']
j = mag['Voltage Y (V)']
i = mag['Voltage X (V)']

############################################ This is the sample rate we used in testing ########################################
fs = 5120
############################################# If you used something else, put it here ##########################################

## The following cell displays the time series of the recorded data

In [ ]:
############################################ You can define the y limits #######################################################

#ylimmax = 
#ylimmin =

####################### You can change the linestyle/order in which they are ploted for ease of viewing ########################
plt.figure(1, figsize = (19, 8))

plt.plot(times,i, label = 'X', color = 'red', linewidth = 1.5)

plt.title("Time Series X-Dir", fontweight = 'bold', fontsize = 25)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)

plt.xlim(0,times[len(times)-1])
#plt.ylim(ylimmin,ylimmax)

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)

################################################################################################################################

plt.figure(2, figsize = (19, 8))

plt.plot(times,j, label = 'Y', color = 'blue', linewidth = 1.5)

plt.title("Time Series Y-Dir", fontweight = 'bold', fontsize = 25)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)

plt.xlim(0,times[len(times)-1])
#plt.ylim(ylimmin,ylimmax)

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)

################################################################################################################################

plt.figure(3, figsize = (19, 8))

plt.plot(times,k, label = 'Z', color = 'black', linewidth = 1.5)

plt.title("Time Series Z-Dir", fontweight = 'bold', fontsize = 25)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)

plt.xlim(0,times[len(times)-1])
#plt.ylim(ylimmin,ylimmax)

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)

################################################################################################################################
'''
plt.figure(4, figsize = (19, 8))

plt.plot(times,noi, label = 'Noise', color = 'yellow', linewidth = 1.5)

plt.title("Time Series Noise", fontweight = 'bold', fontsize = 25)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)

plt.xlim(0,times[len(times)-1])
#plt.ylim(ylimmin,ylimmax)

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)
'''
#plt.savefig("**place_img_name_here.format**", bbox_inches='tight')

## If you want to look at a specific time along the time series, you can use the slider plot below. 

### These variables are used to adjust the how many seconds are displayed and the step length

In [ ]:
win = 15  ## sec displayed
stepy = 4  ## sec moved

## Beware, there is a slight time delay when slidering

## If the slider is not workering move to the next cell

## If you want to look at a specific time along the time series, you can use the slider plot below. Beware, there is a slight time delay


## If the slider is not workering move to the next cell

In [ ]:
Min = min(k.min(), j.min(), i.min())  ########################## This is to find the window height #############################
Max = max(k.max(), j.max(), i.max())

##################################### This function is what makes the slider run ###############################################
######################################### Parameters can be changed as needed ##################################################

def plot_with_slider(xlim):
    
    plt.figure(figsize=(19, 8))   ## dpi breaks slider
    
    plt.plot(times, i, linewidth = 1.5, label = 'X', color = 'red')
    plt.plot(times, k, linewidth = 1.5, label = 'Z', color = 'black')
    plt.plot(times, j, linewidth = 1.5, label = 'Y', color = 'blue')
    
    plt.xlim(xlim, xlim + win) ######################## This is how long the window is (CAN be changed) ########################
    plt.ylim(Min, Max)        #################### This adds a buffer to the window height (CAN be changed) ####################
    
    plt.legend(loc = 'upper right', fontsize = 15)
    plt.title("Slider Time Series", fontweight = 'bold', fontsize = 25)
    plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
    plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)
    plt.yticks(fontsize = 20, fontweight = "bold")
    plt.xticks(fontsize = 20, fontweight = "bold")
    plt.grid(True)
    plt.show()

######################################## This is where the slider plot gets plotted ############################################
#### min,max = start,end ; step is the increment it slides on ; value is where it starts ; layout is how long the slider is ####
############################################### step and layout CAN be changed #################################################

interact(plot_with_slider, xlim = FloatSlider(min = 0, max = (times[len(times)-1]), step = stepy, value = 0, 
                                              layout=Layout(width='980px')))


## If the slider is not working, you can use this cell to manually change the limits to look at a specific area of the time series.

In [ ]:
plt.figure(figsize = (19, 8))

####################### You can change the linestyle/order in which they are ploted for ease of viewing ########################

plt.plot(times,i, label = 'X', color = 'red', linewidth = 1.5)
#plt.plot(times,k, label = 'Z', color = 'black', linewidth = 1.5)
#plt.plot(times,j, label = 'Y', color = 'blue', linewidth = 1.5)
#plt.plot(times,noise, label = 'Noise', color = 'yellow', linewidth = 1.5)

plt.legend(loc = 'upper right', fontsize = 15)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)

#plt.xlim()
#plt.ylim()

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)
#plt.savefig("**place_img_name_here.format**", bbox_inches='tight')

## The following cells are used to plot spectrograms for each direction

### These variables are to set the color magnitude limits of the spectrogram, the y limits, and the fft length

In [ ]:
cmin = 0.0001 ## 0.0001 is the lowest it will go
cmax = 0.009 ## Increase to make the plot more blue

ymax = 65 ## In terms of frequency
ymin = 55

###################################################### fft length ##############################################################
fft_len = 128 ## In terms of sec

################################################# Precent FFT Overlap ##########################################################
olap = 50 # 50% fft overlap

In [ ]:
################################################################################################################################
###################### The spectrograms are calculated with signal.spectrogram from SciPy ######################################
################################################################################################################################
warnings.simplefilter('ignore')

fk, tk, Sxxk = signal.spectrogram(k, fs, window= 'hamming', nperseg=round(fs *fft_len), noverlap= round(fs /(olap *0.01)) )
fj, tj, Sxxj = signal.spectrogram(j, fs, window= 'hamming', nperseg=round(fs *fft_len), noverlap= round(fs /(olap *0.01)) )
fi, ti, Sxxi = signal.spectrogram(i, fs, window= 'hamming', nperseg=round(fs *fft_len), noverlap= round(fs /(olap *0.01)) )
f, t, Sxx = signal.spectrogram(noi, fs, window= 'hamming', nperseg=round(fs *fft_len), noverlap= round(fs /(olap *0.01)) )


fig = plt.figure(figsize = (20,24))

gs = gridspec.GridSpec(4,1, height_ratios = [1,1,1,1], hspace = 0.35)

####################################################### Channel Z ##############################################################

axis1 = fig.add_subplot(gs[0,0])

m0 = axis1.pcolormesh(tk, fk, np.sqrt(Sxxk), shading='gouraud') ############### This will autoscale the plot ###################
                                                                ## If you want to change the limits, uncomment the line below ##
                                                                ########### Be sure to comment out the line above ##############

#m0 = axis1.pcolormesh(tk, fk, np.sqrt(Sxxk), shading='gouraud', vmin = cmin, vmax = cmax)

axis1.set_title("Z Direction", fontweight = 'bold' , fontsize = 20)
axis1.set_xlabel('Time (s)', fontweight = 'bold' , fontsize = 18)
axis1.set_ylabel('Frequency (Hz)', fontweight = 'bold' , fontsize = 18)
axis1.tick_params(labelsize=16)

cbar0 = fig.colorbar(m0, label='Intensity [m/s/Hz^0.5]', pad =0.02)
cbar0.ax.tick_params(labelsize=14)
cbar0.set_label(label='Intensity [m/s/Hz^0.5]' ,weight='bold', fontsize = 14)

axis1.set_ylim(ymin, ymax)

####################################################### Channel Y ##############################################################

axis2 = fig.add_subplot(gs[1,0])

m1 = axis2.pcolormesh(tj, fj, np.sqrt(Sxxj), shading='gouraud') ############### This will autoscale the plot ###################

#m1 = axis2.pcolormesh(tj, fj, np.sqrt(Sxxj), shading='gouraud', vmin = cmin, vmax = cmax)

axis2.set_title("Y Direction", fontweight = 'bold' , fontsize = 20)
axis2.set_xlabel('Time (s)', fontweight = 'bold' , fontsize = 18)
axis2.set_ylabel('Frequency (Hz)', fontweight = 'bold' , fontsize = 18)
axis2.tick_params(labelsize=16)

cbar1 = fig.colorbar(m1, pad =0.02)
cbar1.ax.tick_params(labelsize=14)
cbar1.set_label(label='Intensity [m/s/Hz^0.5]' ,weight='bold', fontsize = 14)

axis2.set_ylim(ymin, ymax)

####################################################### Channel X ##############################################################

axis3 = fig.add_subplot(gs[2,0])

m2 = axis3.pcolormesh(ti, fi, np.sqrt(Sxxi), shading='gouraud') ############### This will autoscale the plot ###################

#m2 = axis3.pcolormesh(ti, fi, np.sqrt(Sxxi), shading='gouraud', vmin = cmin, vmax = cmax)

axis3.set_title("X Direction", fontweight = 'bold' , fontsize = 20)
axis3.set_xlabel('Time (s)', fontweight = 'bold' , fontsize = 18)
axis3.set_ylabel('Frequency (Hz)', fontweight = 'bold' , fontsize = 18)
axis3.tick_params(labelsize=16)

cbar2 = fig.colorbar(m2, pad =0.02)
cbar2.ax.tick_params(labelsize=14)
cbar2.set_label(label='Intensity [m/s/Hz^0.5]' ,weight='bold', fontsize = 14)

axis3.set_ylim(ymin, ymax)

###################################################### Noise Channel ###########################################################
'''
axis4 = fig.add_subplot(gs[3,0])

axis4.set_title("Noise Direction", fontweight = 'bold' , fontsize = 20)

m3 = axis4.pcolormesh(t, f, np.sqrt(Sxx), shading='gouraud', vmin = cmin, vmax = cmax)

axis3.set_title("Electronic Noise", fontweight = 'bold' , fontsize = 20)
axis3.set_xlabel('Time (s)', fontweight = 'bold' , fontsize = 18)
axis4.set_ylabel('Frequency (Hz)', fontweight = 'bold' , fontsize = 18)
axis3.tick_params(labelsize=16)

cbar3 = fig.colorbar(m3, pad = 0.02)
cbar3.ax.tick_params(labelsize=14)
cbar3.set_label(label='Intensity [m/s/Hz^0.5]' ,weight='bold', fontsize = 14)


axis4.set_ylim(ymin, ymax)

#plt.savefig('**place_img_name_here.format**', bbox_inches='tight')
'''
################################################################################################################################
#################################### These may take a while to plot, since they are so dense ###################################

## If you want to look at spectra of specific parts of the time series, you can set the time limits here

## If not, skip the following two cells

In [ ]:
################################### Enter how long of a time segment you'd like to look at #####################################
start = 0  ## In terms of seconds
end = 100

## These are loops to index the dataframes

In [ ]:
a = 0 ################################################# Variables for loops ####################################################
b = 0 

################################################################################################################################
############################# This loop is for selecting only the start-end data from the file #################################

for a in range(0, len(times)): ############### These are to find the index values of start,end from the data file ##############
    if round(times[a]) == start:
        alpha = samples[b]
        break
        
for b in range(0, len(times)):
    if round(times[b]) == end:
        beta = samples[b]
        break

################################################### The data gets sliced #######################################################

t = times[alpha:beta]
#ns = noise[alpha:beta]
k1 = k[alpha:beta]
j1 = j[alpha:beta]
i1 = i[alpha:beta]

## This is where the ASD is calculated and plotted

In [ ]:
##################################################### Plot limits ##############################################################

y_max = 10e-1
y_min = 10e-4 

x_max = 2500
x_min = 5e-2

###################################################### fft length ##############################################################
fft_length = 128

################################################# Precent FFT Overlap ##########################################################
overlap = 50 # 50% fft overlap

In [ ]:
################################################################################################################################
######################################### The PSD is calculated via welch from scipy ###########################################
################################################################################################################################
warnings.simplefilter('ignore')

########### If you don't want to look at the whole time series, you can add '1' to the variables, i.e k1, j1, i1 ###############

f_k, Pxx_den_k = signal.welch(k, fs, window= 'hamming', nperseg= (fs *fft_length), noverlap= round(fs /(overlap *0.01)) )
f_j, Pxx_den_j = signal.welch(j, fs, window= 'hamming', nperseg= (fs *fft_length), noverlap= round(fs /(overlap *0.01)) )
f_i, Pxx_den_i = signal.welch(i, fs, window= 'hamming', nperseg= (fs *fft_length), noverlap= round(fs /(overlap *0.01)) )

################# nfft is how long the fft bin lengths, noverlap is bin overlap, window is the window used #####################
##################### We used 100 sec bins, 0.5 sec overlap, and hamming window, these can be changed ##########################
################################################ # DO NOT CHANGE/REMOVE 'fs' # #################################################


ak = np.sqrt(Pxx_den_k)
aj = np.sqrt(Pxx_den_j) ################################ This turns the PSD into an ASD ########################################
ai = np.sqrt(Pxx_den_i)


l_k = np.log(ak)
l_j = np.log(aj) ################# The log of each ASD must be found in order to find the peaks/frequences #####################
l_i = np.log(ai)


peaks_k, _ = signal.find_peaks(l_k, prominence = 4.5) ############# This is where the frequencies are found ####################
peaks_j, _ = signal.find_peaks(l_j, prominence = 4.5) ############## If some are missed or there are extra #####################
peaks_i, _ = signal.find_peaks(l_i, prominence = 4.5) ######### you will need to change the prominence variable ################


################################################################################################################################
##################################################### Plots peaks ##############################################################

plt.figure(figsize = (20, 8)) 


plt.scatter(f_k[peaks_k], ak[peaks_k] ** 0.5, s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5, label = 'Signal Peaks')
plt.scatter(f_j[peaks_j], aj[peaks_j] ** 0.5, s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5)
plt.scatter(f_i[peaks_i], ai[peaks_i] ** 0.5, s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5)


###################################################### PLots data ##############################################################

plt.yscale('log')
plt.xscale('log')

plt.plot(f_k, ak ** 0.5, color = 'black', linewidth = 1.5, label = 'Z Channel')
plt.plot(f_j, aj ** 0.5, color = 'blue', linewidth = 1.5, label = 'Y Channel')
plt.plot(f_i, ai ** 0.5, color = 'red', linewidth = 1.5, label = 'X Channel')

plt.legend(loc = "upper right", fontsize = 15)

plt.title("Magnetic Data ASD", fontweight = 'bold', fontsize = 25)
plt.xlabel("Frequency [Hz]", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude [V/Hz^0.5]", fontweight = "bold", fontsize = 20)

plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")

plt.ylim(y_min,y_max)
plt.xlim(x_min,x_max)

plt.grid(True, which="both", ls="-")

#plt.savefig('**place_img_name_here.format**', bbox_inches='tight')

## The following cell will plot each of direction signal in their own respective plots
### If you don't want/need to look at them individually, you don't have to run it

In [ ]:
################################################################################################################################
############################# This gets plotted as a multipanel plot via gridspec and subplot ##################################
################################################################################################################################

fig = plt.figure(figsize = (20,24))

gs = gridspec.GridSpec(3,1, height_ratios = [1,1,1], hspace = 0.35)

####################################################### Channel Z ##############################################################

axis1 = fig.add_subplot(gs[0,0])

axis1.set_title("Z Direction", fontweight = 'bold' , fontsize = 20)

axis1.set_yscale('log')
axis1.set_xscale('log')

axis1.scatter(f_k[peaks_k], ak[peaks_k], s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5, label = 'Signal Peaks')
axis1.plot(f_k, ak, color = 'black', linewidth = 1.5, label = 'Channel Z')

axis1.set_xlabel("Frequency [Hz]", fontweight = "bold", fontsize = 18)
axis1.set_ylabel("Amplitude [V/Hz^0.5]", fontweight = "bold", fontsize = 18)
axis1.tick_params(labelsize=16)

axis1.set_ylim(y_min,y_max)
axis1.set_xlim(x_min,x_max)
axis1.grid(True, which="both", ls="-")

######################################################## Channel Y #############################################################

axis2 = fig.add_subplot(gs[1,0])

axis2.set_title("Y Direction", fontweight = "bold", fontsize = 20)

axis2.set_yscale('log')
axis2.set_xscale('log')

axis2.scatter(f_j[peaks_j], aj[peaks_j], s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5)
axis2.plot(f_j, aj, color = 'blue', linewidth = 1.5, label = 'Channel Y')

axis2.set_xlabel("Frequency [Hz]", fontweight = "bold", fontsize = 18)
axis2.set_ylabel("Amplitude [V/Hz^0.5]", fontweight = "bold", fontsize = 18)
axis2.tick_params(labelsize=16)

axis2.set_ylim(y_min,y_max)
axis2.set_xlim(x_min,x_max)
axis2.grid(True, which="both", ls="-")

######################################################## Channel X #############################################################

axis3 = fig.add_subplot(gs[2,0])

axis3.set_title("X Direction", fontweight = "bold", fontsize = 20)

axis3.set_yscale('log')
axis3.set_xscale('log')

axis3.scatter(f_i[peaks_i], ai[peaks_i], s = 100, color = 'lime', marker = 'x', 
            linewidths = 2.5)
axis3.plot(f_i, ai, color = 'red', linewidth = 1.5, label = 'Channel X')

axis3.set_xlabel("Frequency [Hz]", fontweight = "bold", fontsize = 18)
axis3.set_ylabel("Amplitude [V/Hz^0.5]", fontweight = "bold", fontsize = 18)
axis3.tick_params(labelsize=16)

axis3.set_ylim(y_min,y_max)
axis3.set_xlim(x_min,x_max)
axis3.grid(True, which="both", ls="-")